In [1]:
## Data Preparation
# List of functions for loading Belgium Traffic Sign Data Set for (Training and Testing)
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [2]:
## Import libraries need to be imported, Dont forget to update requirements.txt!
import os
import sys
IntialiseEnv()
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import random
import matplotlib.pyplot as plt
import import_ipynb
from Pre_Processing.Pre_Processing import FeatureExtraction
from Pre_Processing.Pre_Processing import augment_image, motion_blur_horizontal_vertical, motion_blur,another_augment

importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Pre_Processing\Segmentation.ipynb


In [3]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(60,60))
    return img

In [4]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [10]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(datasetDir,_dir):
    train_category_dir= os.path.join(datasetDir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

In [6]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(60,60))
    return img

In [33]:
## Training DataSet Directory
training_belgium_dataset_dir = '../DataSet/BelgiumTSC_Training/Training/'

def LoadBelgiumTrainDataSet():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_belgium_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(training_belgium_dataset_dir, _dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array

In [47]:
test_belgium_dataset_dir='../DataSet/BelgiumTSC_Testing/Testing/'
def LoadBelgiumTestDataSet():
    test_image_array=[]
    test_image_labels=[]

    for _dir in os.listdir(test_belgium_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        test_category_dir, test_data_frame= getDirAndDataFrame(test_belgium_dataset_dir, _dir)
        # Get ClassID for entire category
        if len(test_data_frame['ClassId']) == 0:
            print("Skipping Directory "+ _dir + " No test images available")
            continue
            
        dir_img_label=test_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(test_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            test_image_array.append(img_features)
            #Populate associated labels
            test_image_labels.append(dir_img_label)
            
    test_image_array=np.stack(test_image_array, axis=0)
    return test_image_array, test_image_labels

In [ ]:
weak_class_labels_str=['00000' , '00002' ,'00003' ,'00004' ,'00005' ,'00006' ,'00008' ,'00009' ,'00010' ,'00011' ,'00012' ,'00015' ,'00016' ,'00023' ,'00026' ,'00027','00029' ,'00030' ,'00033' ,'00036' ,'00042','00043' ,'00044' ,'00048' ,'00049' ,'00050' ,'00051' ,'00052' ,'00055','00058','00060' ]

In [ ]:
def LoadBelgiumTrainDataSetWithAugmentation():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_belgium_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(training_belgium_dataset_dir,_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            #### for agumentation 

            if _dir in weak_class_labels_str:    
                random_number_probability=random.uniform(0, 1)

                if random_number_probability>=0.1:
                    agumented_image=augment_image(img,0.7,11,5,2)
                    img_features_= np.array(FeatureExtraction(agumented_image))[:,:,np.newaxis]
                    train_labels_array.append(dir_img_label)
                    train_image_array.append(img_features_)
                    agumented_image=another_augment(img)
                    img_features_= np.array(FeatureExtraction(agumented_image))[:,:,np.newaxis]
                    train_labels_array.append(dir_img_label)
                    train_image_array.append(img_features_)


                if random_number_probability>=0.1:
                    # img = FeatureExtraction(img)
                    blurred_img=motion_blur(img) 
                    train_labels_array.append(dir_img_label)
                    img_features_= np.array(FeatureExtraction(blurred_img))[:,:,np.newaxis] 
                    train_image_array.append(img_features_)
            else:
                random_number_probability=random.uniform(0, 1)

                if random_number_probability<=0.3:
                    agumented_image=augment_image(img,0.7,11,5,2)
                    img_features_= np.array(FeatureExtraction(agumented_image))[:,:,np.newaxis]
                    train_labels_array.append(dir_img_label)
                    train_image_array.append(img_features_)
                    agumented_image=another_augment(img)
                    img_features_= np.array(FeatureExtraction(agumented_image))[:,:,np.newaxis]
                    train_labels_array.append(dir_img_label)
                    train_image_array.append(img_features_)


                if random_number_probability<=0.3:
                    # img = FeatureExtraction(img)
                    blurred_img=motion_blur(img) 
                    train_labels_array.append(dir_img_label)
                    img_features_= np.array(FeatureExtraction(blurred_img))[:,:,np.newaxis] 
                    train_image_array.append(img_features_)

    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array